**Классификация изображений с помощью сверточных нейронных сетей**

В данном задании Вам необходимо разработать архитектуру сверточной ИНС, обеспечивающую наибольшую точность при ограничении на количество операций (FLOPs <= 0.707e6).
Заготовка кода для выполнения задания приведена выше. Вашей задачей будет заполнить пропущеные места, которые отмечены ключевым словом *None*.
Необходимая точность (accuracy) сети на датасете CIFAR100 - 30%
Желаемая точность (accuracy) сети на датасете CIFAR100 - 45%

In [1]:
!pip install keras-flops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import tensorflow as tf
from keras_flops import get_flops

In [3]:
# Глобальные константы
CLASSES       = 100
BATCH_SIZE    = 128
LEARNING_RATE = 1e-2

In [4]:
# Выполните загрузку модели
from keras.datasets import cifar100
(X_train, y_train), (X_val, y_val) = cifar100.load_data()

169001437/169001437 [==============================] - 2s 0us/step


In [5]:
# Преобразуйте метки классов в one_hot формат

from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train, 100)
y_val = np_utils.to_categorical(y_val, 100)

In [6]:
y_train.shape
y_val.shape

(10000, 100)

In [7]:
# убедитесь, что данная ячейка выполняется без ошибок
assert X_train.shape == (50000, 32, 32, 3)
assert X_val.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 100)
assert y_val.shape == (10000, 100)

In [24]:
# Задайте архитектуру модели
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=[32,32,3]),
    tf.keras.layers.Conv2D(16, kernel_size=(4,4),strides=(2,2),padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPool2D(pool_size=(4,4)),
    tf.keras.layers.Conv2D(16, kernel_size=(3,3), padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    #tf.keras.layers.MaxPool2D(pool_size=(2,2)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(300),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(CLASSES),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('softmax')
])

In [25]:
# вычисление количества операций
flops = get_flops(model, batch_size=1)
print(f"FLOPs: {(flops / 1e6):.4f}e6")

FLOPs: 0.7020e6


In [26]:
# вывод краткой информации о модели
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 16, 16, 16)        784       
                                                                 
 batch_normalization_16 (Bat  (None, 16, 16, 16)       64        
 chNormalization)                                                
                                                                 
 activation_16 (Activation)  (None, 16, 16, 16)        0         
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 4, 4, 16)         0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 4, 4, 16)          2320      
                                                                 
 batch_normalization_17 (Bat  (None, 4, 4, 16)        

In [28]:
# параметры данной ячейки могут быть изменены для получения более высокой точности
model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=LEARNING_RATE, decay_steps=1000, decay_rate=0.4)
    ),
    #optimizer=tf.keras.optimizers.SGD(
    #    momentum=0.5,
    #    learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.01, decay_steps=1000, decay_rate=0.5)
    #),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [29]:
# обучения модели
model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_val, y_val),
    batch_size=BATCH_SIZE,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(filepath="{epoch:02d}-{val_accuracy:.2f}.hdf5", save_best_only=True),
        
    ],
    use_multiprocessing=True,
    workers=8,
    epochs=200
)

Epoch 1/200
391/391 [==============================] - 3s 7ms/step - loss: 3.4660 - accuracy: 0.1828 - val_loss: 3.2958 - val_accuracy: 0.2195
Epoch 2/200
391/391 [==============================] - 2s 6ms/step - loss: 2.9465 - accuracy: 0.2705 - val_loss: 3.1810 - val_accuracy: 0.2278
Epoch 3/200
391/391 [==============================] - 2s 5ms/step - loss: 2.7353 - accuracy: 0.3124 - val_loss: 2.8242 - val_accuracy: 0.3008
Epoch 4/200
391/391 [==============================] - 2s 6ms/step - loss: 2.5957 - accuracy: 0.3383 - val_loss: 2.6177 - val_accuracy: 0.3467
Epoch 5/200
391/391 [==============================] - 2s 5ms/step - loss: 2.4925 - accuracy: 0.3612 - val_loss: 2.6205 - val_accuracy: 0.3503
Epoch 6/200
391/391 [==============================] - 2s 6ms/step - loss: 2.4131 - accuracy: 0.3773 - val_loss: 2.4949 - val_accuracy: 0.3701
Epoch 7/200
391/391 [==============================] - 2s 6ms/step - loss: 2.3567 - accuracy: 0.3881 - val_loss: 2.4604 - val_accuracy: 0.3788